In [76]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import base64
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
import plotly.graph_objects as go


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
from bson.json_util import dumps, LEGACY_JSON_OPTIONS
import json


#import CRUD.py with the AAC animal shelter database
from CRUD import AnimalShelter



###########################
# Data Manipulation / Model
###########################

def get_df_from_query(query):
    # create dataframe given records from database
    df = pd.DataFrame(list(shelter.read(query)))
    df = df.iloc[:, 1:]
    return df

#Water Rescue Query
wr_query = {"breed":{"$in":["Labrador Retriever Mix", "Chesa Bay Retr Mix","Newfoundland Mix"]},"sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{"$gt":26,"$lt":156}}
#Mountain Rescue Query
mr_query = {"breed":{"$in":["German Shepherd Mix","Alaska Malamute Mix","Old English Sheepdog", "Siberian Husky Mix", "Rottweiler"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gt":26,"$lt" : 156}}
#Disaster Rescue Query
dr_query = {"breed":{"$in":["Doberman Pinscher","German Shepherd Mix","Golden Retriever","Bloodhound", "Rottweiler"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gt":20,"$lt":300}}

#Credentials of db
username = "aacuser"
password = "abcuser1"
port = 38690
#Connect to db
shelter = AnimalShelter(username, password, port)

#Get all records from db
df = get_df_from_query({})
#Create pie chart
pie_fig = px.pie(df.to_dict('records'), names='breed')

#name of image
image_filename = 'logo.png'
#base64 encoding of image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('ProjectTwo')

app.layout = html.Div(
    [
        html.Div(id='hidden-div', style={'display':'none'}),
        html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
        html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image))),
        html.Hr(),
        #Filter radio buttons
        dcc.RadioItems(
           id='filter-id',
           options=[
               {'label': 'Water Rescue', 'value': 'WR'},
               {'label': 'Mountain Rescue', 'value': 'MR'},
               {'label': 'Disaster Rescue', 'value': 'DR'},
               {'label': 'Reset', 'value': 'RR'}
           ],
           value='RR',
           labelStyle={'display': 'inline-block'}
        ),
        html.Br(),
        #Data table
        dash_table.DataTable(
            id='datatable-id',
            #columns of table
            columns=[
                {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
            ],
            #rows of table
            data=df.to_dict('records'),
            #style of table
            style_data={
                'whiteSpace': 'normal',
                'height': 'auto',
                'lineHeight': '15px'
            },
            #amount of items per page
            page_size=10,
            filter_action="native"
        ),
        html.Br(),
        html.Hr(),
        #Pie Chart
        dcc.Graph(id="pie-chart", figure=pie_fig),
        #Map
        html.Div(
            id='map-id',
        ), 
        #unique identifier
        html.H6("Jacob Valdiviez - 681995")
    ]
)

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it

#Call back for selection of data table cells
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
      return [{'if': { 'column_id': i },'background_color': '#D2F3FF'   } for i in selected_columns]

#Call back for updating map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    #children of Map
    c = []
    c.append(dl.TileLayer(id="base-layer-id"))
    #create 10 markers
    for x in range(0, 10):
        #add marker
        c.append(
            #add lat, long
            dl.Marker(position=[dff.iloc[x,13],dff.iloc[x,14]], children=[
                #tool tip is breed of dog
                dl.Tooltip(dff.iloc[x,4]),
                #popup has name of animal and status of animal
                dl.Popup([
                    html.H1(dff.iloc[x,9]),
                    html.P(dff.iloc[x,11])
                ])
            ])
        )
    #return the map
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=c)
    ]
#call back for updating pie chart and data table after filter
@app.callback(
    [Output('pie-chart','figure'),
    Output('datatable-id', 'data')],
    [Input('filter-id', 'value')])
def filter_data(filter_value):
    query = {}
    #Reset button
    if filter_value == 'RR':
        query = {}
    #Water button
    elif filter_value == 'WR':
        query = wr_query
    #Mountain button
    elif filter_value == 'MR':
        query = mr_query
    #Disaster button
    elif filter_value == 'DR':
        query = dr_query
    #Update data with query
    df = get_df_from_query(query)
    data = df.to_dict('records')
    #Update piechart with new data
    fig = px.pie(data, names='breed')
    
    return fig, data
    
app

connected!
